# Simulation interface
> Main simulation class and command line client

Create a class which:

* parses the configuration file
* gets the channels that the user wants to simulate
* and the number of splits
* loops through these and returns/writes the maps

In [ ]:
# default_exp cli

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export

import toml
import healpy as hp
import numpy as np
import h5py
from path import Path
import logging as log

from s4refsimtool.core import get_telescope, parse_channels

from s4refsimtool import __version__

from s4refsimtool.foregrounds import load_sky_emission
from s4refsimtool.atmosphere import load_atmosphere, get_telecope_years
from s4refsimtool.noise import load_noise

In [ ]:
# exports


class S4RefSimTool:
    def __init__(self, config, output_folder="output"):
        """Simulate CMB-S4 maps based on the experiment configuration

        Parameters
        ----------
        config : filename
            CMB-S4 configuration stored in a TOML file
            see for example s4_reference_design.toml in the repository
        output_folder : str or Path
            Output path
        """
        self.config = toml.load(config)
        self.output_filename_template = (
            "cmbs4_{tag}_KCMB_{telescope}-{band}_nside{nside}_{split}_of_{nsplits}.fits"
        )
        self.output_folder = Path(output_folder)

    def run(
        self, channels="all", sites=["Pole", "Chile"], nsplits=1, write_outputs=True
    ):
        """Run the simulation

        Parameters
        ----------
        channels : str or list[str]
            list of channel tags, e.g.
            * ["LFS1", "LFS2"] or
            * "SAT" or "LAT"
            * "all" (default)
        site : list[str]
            ['Pole'] or ['Chile'], by default ["Pole", "Chile"]
        nsplits : int
            Number of splits, 1 generates only full mission
            2-7 generates the full mission map and then the requested number
            of splits scaled accordingly. E.g. 7 generates the full mission
            map and 7 equal (yearly) maps
        write_outputs : bool
            if True write the outputs to disk, if False return them (not implemented yet)
        """
        assert (
            nsplits < 8
        ), "We currently only have 7 independent realizations of atmosphere and noise"
        for site in sites:
            for channel in parse_channels(channels):
                if get_telecope_years(self.config, site, channel) == 0:
                    continue
                log.info(f"Simulate channel {channel} at {site}")
                sky_emission = load_sky_emission(
                    self.config["sky_emission"], site, channel
                )
                for split in range(nsplits):
                    output_map = load_atmosphere(
                        self.config, site, channel, realization=split
                    )
                    output_map += load_noise(
                        self.config, site, channel, realization=split
                    )
                    if split > 0:
                        output_map *= np.sqrt(nsplits)
                    output_map += sky_emission
                    if write_outputs:
                        output_filename = self.output_filename_template.format(
                            nside=hp.npix2nside(len(output_map[0])),
                            telescope=get_telescope(channel),
                            band=channel,
                            tag="sky_atmosphere_noise",
                            split=max(
                                1, split
                            ),  # split=0 is full mission and we want 1
                            nsplits=nsplits,
                        )
                        log.info(f"Writing {output_filename}")
                        hp.write_map(
                            self.output_folder / output_filename,
                            output_map,
                            column_units="K_CMB",
                            extra_header={"tool_version": __version__},
                            coord="Q",
                            overwrite=True,
                        )
                    else:
                        raise NotImplementedError("Only writing FITS output for now")

In [ ]:
log.basicConfig(level = log.INFO)

In [ ]:
sim = S4RefSimTool("s4_reference_design.toml")

In [ ]:
%ls /global/cscratch1/sd/keskital/s4sim/reference_tool/out/00000000/*atmo*

In [ ]:
sim.run(channels="LFL1", sites=["Chile"])

In [ ]:
%matplotlib inline

In [ ]:
output_map = hp.read_map(
    "output/cmbs4_sky_atmosphere_noise_KCMB_LAT-LFL1_nside4096_1_of_1.fits", (0,1,2))

In [ ]:
hp.mollview(output_map[0], min=-1e-3, max=1e-3, unit="K", title="Total I")

In [ ]:
hp.mollview(output_map[1], min=-1e-3, max=1e-3, unit="K", title="Total Q")

In [ ]:
hp.mollview(output_map[2], min=-1e-3, max=1e-3, unit="K", title="Total U")